# Install InSynth

First, we install the InSynth library using pip:

In [12]:
!pip install insynth==1.0.2

# Download Data (ImageNet)

Then, download the ImageNetV2 dataset and use the first 100 samples.

In [13]:
import tensorflow_datasets as tfds
from PIL import Image

dataset = tfds.load('imagenet_v2', split='test', shuffle_files=False).take(100)

Then, a generator is created which resizes the images to 299x299 pixels for the XCeption model:

In [14]:
x_test_generator = lambda:(Image.fromarray(sample['image'].numpy()).resize((299, 299)).convert('RGB') for sample in
                    dataset)

Then, the target variables are stored in an array:

In [15]:
y_test = [sample['label'].numpy() for sample in dataset]

In [16]:
print(f'Number of samples in the dataset: {len(y_test)}')

Number of samples in the dataset: 100


# Download Model (XCeption)

We use a pretrained XCeption model:

In [17]:
from tensorflow import keras

model = keras.applications.Xception()

print(model.summary())

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

# Run Robustness Test

To run the robustness test, we use the `BasicImageRunner` class and pass the relevant perturbators and calculators:

In [18]:
from insynth.perturbators.image import ImageNoisePerturbator, ImageOcclusionPerturbator
from insynth.calculators import NeuronCoverageCalculator
from insynth.runners.runner import BasicImageRunner
import numpy as np

runner = BasicImageRunner(perturbators=[ImageNoisePerturbator(p=1.0), ImageOcclusionPerturbator(p=1.0)], 
                          coverage_calculators =[NeuronCoverageCalculator(model)],
                          dataset_x=x_test_generator, 
                          dataset_y=y_test, 
                          model=model,
                          pre_predict_lambda=lambda sample: keras.applications.xception.preprocess_input(
                                      np.expand_dims(np.array(sample), axis=0)))

Once the runner is instantiated, the test can be run:

In [19]:
report, robustness = runner.run(False, 'output/imagenet/xception')

Processing Original Dataset...: 100it [03:22,  2.02s/it]
Applying Perturbators...: 0it [00:00, ?it/s]
Running on Samples...: 0it [00:00, ?it/s]
Running on Samples...: 1it [00:01,  1.79s/it]
Running on Samples...: 2it [00:03,  1.76s/it]
Running on Samples...: 3it [00:05,  1.76s/it]
Running on Samples...: 4it [00:07,  1.75s/it]
Running on Samples...: 5it [00:08,  1.74s/it]
Running on Samples...: 6it [00:10,  1.73s/it]
Running on Samples...: 7it [00:12,  1.74s/it]
Running on Samples...: 8it [00:13,  1.74s/it]
Running on Samples...: 9it [00:15,  1.74s/it]
Running on Samples...: 10it [00:17,  1.75s/it]
Running on Samples...: 11it [00:19,  1.75s/it]
Running on Samples...: 12it [00:21,  1.76s/it]
Running on Samples...: 13it [00:22,  1.76s/it]
Running on Samples...: 14it [00:24,  1.75s/it]
Running on Samples...: 15it [00:26,  1.75s/it]
Running on Samples...: 16it [00:27,  1.75s/it]
Running on Samples...: 17it [00:29,  1.75s/it]
Running on Samples...: 18it [00:31,  1.76s/it]
Running on Samples.

Now, the results can be analysed:

In [20]:
print(f'The model has a robustness score of {robustness}')

The model has a robustness score of 0.8066666666666666


The detailed results can be examined in the `report` dataframe:

In [21]:
report

,acc,macro_f1,macro_rec,macro_prec,micro_f1,micro_rec,micro_prec,NeuronCoverageCalculator,args
Original,0.690000,0.543785,0.545198,0.550847,0.695000,0.690000,0.720000,"{'total_neurons': 62713664, 'covered_neurons':...",NaN
0_ImageNoisePerturbator,0.620000,0.463415,0.462060,0.474255,0.630000,0.620000,0.663333,"{'total_neurons': 62713664, 'covered_neurons':...","{'p': 1.0, 'noise_prob': <scipy.stats._continu..."
1_ImageOcclusionPerturbator,0.180000,0.133333,0.130769,0.138462,0.186667,0.180000,0.200000,"{'total_neurons': 62713664, 'covered_neurons':...","{'p': 1.0, 'strength_prob': <scipy.stats._cont..."
All,0.496667,0.380178,0.379342,0.387855,0.503889,0.496667,0.527778,"{'total_neurons': 62713664, 'covered_neurons':...",NaN
